In [3]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt =PromptTemplate.from_template('A {word} is a')

llm = HuggingFacePipeline(
    repo_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 250},
)

chain = prompt | llm

result = chain.invoke(
    {
        'word':'potato'
    }
)

print(result)


/Users/kimdonggyu/Desktop/Study/AI/FULLSTACK-GPT/env/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


A potato is a starchy tuber from the Solanum tuberosum plant, native to South America. It is a common food item worldwide, used in a variety of dishes such as fries, chips, mashed potatoes, and baked potatoes. Potatoes are rich in carbohydrates, fiber, and various vitamins, particularly Vitamin C. They come in many different shapes, sizes, and colors, such as the common brown russet, red, and purple varieties.
